In [20]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools
import geopandas as gpd

In [21]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [22]:
# simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

# gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [23]:
ward_df = gpd.read_file('/Volumes/Data2/RST/notebook/GIS/wards_no-island.geojson')

ward_df = ward_df[['AREA_SHORT_CODE', 'geometry']]
ward_df = ward_df.to_crs('epsg:26917')
ward_df['area'] = ward_df['geometry'].area/1000000

ward_df = ward_df.rename(columns = {'AREA_SHORT_CODE':'ward'})

In [24]:
ward_int = pd.read_csv('/Volumes/Data2/RST/notebook/GIS/int_wards.csv')

#ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')

ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [25]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0
        

In [27]:
period_list = [['EM', [3,4]], ['AM', [7,8]], ['MD', [11,12]], ['PM', [16,17]], ['EV', [20,21]]]
df_list = []
for i in period_list:
    period = i[0]
    

    G = nx.read_gexf('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.gexf')
    graph_data = pd.read_csv('/Volumes/Data2/RST/notebook/networks/' + period +'-RM-16.csv')



    stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

    route_dir = stop_times_od[stop_times_od['hr_o'].isin(i[1])]
    route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()

    route_dir = route_dir.append(route_dir.rename(columns = {'INT_ID_o':'INT_ID_d', 'INT_ID_d':'INT_ID_o'})[['INT_ID_o', 
                                                                                'INT_ID_d', 'route_short_name']])

    route_dir = route_dir.drop_duplicates()


    route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
    route_dir = route_dir.merge(graph_data)

    result_list = []
    ward_n = []

    for ward in range(1,26,1):

        G_ward = G.copy()


        G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

        N = G_ward.size()

        #find cycles
        cycle_list = list(nx.cycle_basis(G_ward))
        print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

        temp_list = list(map(func, cycle_list))


        result_list.append(temp_list)

        if len(cycle_list) == 0:
            result_list.append([(ward, 0, 0, 0)])

        ward_n.append([ward, N])

    combined = list(itertools.chain.from_iterable(result_list))
    result_df = pd.DataFrame.from_records(
        combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
        'ward').sum().reset_index()
    result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))

    result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)
    result_df = result_df.merge(ward_df[['ward', 'area']])

    result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                     right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
        columns = ['AREA_SHORT_CODE'])
    result_df['cycle_density'] = result_df['scaled_cycles']/result_df['area']

    result_df['period'] = period
    
    df_list.append(result_df)
    
    #result_df.to_csv('cycles-' + period + '.csv', index = False)

Ward 1, Size 123 Found 7 cycles
Ward 2, Size 72 Found 4 cycles
Ward 3, Size 109 Found 10 cycles
Ward 4, Size 55 Found 3 cycles
Ward 5, Size 115 Found 11 cycles
Ward 6, Size 91 Found 11 cycles
Ward 7, Size 75 Found 6 cycles
Ward 8, Size 58 Found 2 cycles
Ward 9, Size 66 Found 5 cycles
Ward 10, Size 39 Found 5 cycles
Ward 11, Size 24 Found 0 cycles
Ward 12, Size 24 Found 2 cycles
Ward 13, Size 47 Found 4 cycles
Ward 14, Size 81 Found 8 cycles
Ward 15, Size 39 Found 4 cycles
Ward 16, Size 37 Found 0 cycles
Ward 17, Size 58 Found 4 cycles
Ward 18, Size 59 Found 6 cycles
Ward 19, Size 26 Found 1 cycles
Ward 20, Size 87 Found 11 cycles
Ward 21, Size 72 Found 6 cycles
Ward 22, Size 60 Found 4 cycles
Ward 23, Size 60 Found 3 cycles
Ward 24, Size 43 Found 1 cycles
Ward 25, Size 89 Found 7 cycles
Ward 1, Size 214 Found 29 cycles
Ward 2, Size 174 Found 14 cycles
Ward 3, Size 255 Found 47 cycles
Ward 4, Size 126 Found 22 cycles
Ward 5, Size 207 Found 37 cycles
Ward 6, Size 186 Found 31 cycles
Ward

In [32]:
df = pd.concat(df_list)
df.sort_values(by = 'cycle_density', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,area,ward_name,cycle_density,period
9,10,7,0.909548,4,69,0.006360,9.301089,Spadina-Fort York,0.097789,EV
9,10,7,0.844340,4,70,0.005823,9.301089,Spadina-Fort York,0.090779,MD
12,13,7,0.518868,3,67,0.003733,5.861979,Toronto Centre,0.088514,MD
21,22,22,1.712766,6,127,0.006613,21.395977,Scarborough-Agincourt,0.080051,AM
8,9,12,0.865248,0,101,0.004180,12.092764,Davenport,0.071551,AM
...,...,...,...,...,...,...,...,...,...,...
16,17,0,0.000000,4,58,0.000000,24.416753,Don Valley North,0.000000,EM
18,19,0,0.000000,1,26,0.000000,16.801757,Beaches-East York,0.000000,EM
20,21,0,0.000000,6,72,0.000000,28.201837,Scarborough Centre,0.000000,EM
22,23,0,0.000000,3,60,0.000000,30.433949,Scarborough North,0.000000,EM


In [33]:
df.to_csv('cycles.csv', index = False)